## Repository
Clone https://github.com/jlunder00/OpenGlue

## Data
First, download/unzip data as shown in README.md:

### Steps to prepare MegaDepth dataset for training
1) Create folder `MegaDepth`, where your dataset will be stored.
   ```
   mkdir MegaDepth && cd MegaDepth
   ```
2) Download and unzip `MegaDepth_v1.tar.gz` from official [link](https://www.cs.cornell.edu/projects/megadepth/dataset/Megadepth_v1/MegaDepth_v1.tar.gz).
You should now be able to see `MegaDepth/phoenix` directory.
3) We provide the lists of pairs for training and validation, [link](https://drive.google.com/file/d/1DQl6N1bKEdzlRteCVMS1bWffz0SU-L9x/view?usp=sharing) to download. Each line corresponds to one pair and has the following structure:
```
path_image_A path_image_B exif_rotationA exif_rotationB [KA_0 ... KA_8] [KB_0 ... KB_8] [T_AB_0 ... T_AB_15] overlap_AB
```
`overlap_AB` - is a value of overlap between two images of the same scene, it shows how close (in position transformation) two images are. 

The resulting directory structure should be as follows:
```
MegaDepth/
   - pairs/
   |   - 0000/
   |   |   - sparse-txt/
   |   |   |    pairs.txt
      ...
   - phoenix/S6/zl548/MegaDepth_v1/
   |   -0000/
   |   |   - dense0/
   |   |   |   - depths/
   |   |   |   |   id.h5
                 ...
   |   |   |   - images/
   |   |   |   |   id.jpg
                 ...
   |   |   - dense1/
            ...
      ...
```

The directory structure needs to be the same as shown, even if the data being used is different. To train on different data, create the same directory structure and with data files in the same format. The folder marked 0000/ above represents that everything inside is a part of the same "scene". In the case of the included dataset, images of the same building from different angles/lightings/etc are considered part of the same scene. The format for pairs.txt is shown above, and is needed to add the additional info about pairs of images that OpenGlue uses in it's training. OpenGlue does not include code to generate these files.

## Take a Subset of the Data
OpenGlue includes files specifying which scenes (as in the folder names which each contain a single scene) are to be used for training, testing, and validation, but does not include code for generating new versions of these or for making a smaller sample set.
To take a subset of the data and only use a small amount in training and testing, we need to generate new versions of these files. However, OpenGlue has an intermediate step that is reccommended to be done prior to training, feature extraction, which does not utilize these files and extracts features (keypoints) from the images in the dataset. Feature extraction is highly expensive in terms of disk usage, and takes a long time to do, but speeds up training time. There is not a method implemented in OpenGlue to only preform feature extraction on a subset of the data, but alternatively, we can skip feature extraction as OpenGlue has 2 training methods, one which relies on feature extraction being done prior, and one which does it as training is done. By utilizing the latter, we can have OpenGlue preform feature extraction and training on only a subset of the data, since, although training time would be fastest with a subset and with feature extraction done prior, doing this will make the overall time to go from the beginning of the pipeline to the end faster.

Additionally, store test data in a seperate folder. OpenGlue tests via running inference on 2 jpg images. The images to test on are not provided by OpenGlue, but you can use kaggle data to test (https://www.kaggle.com/competitions/image-matching-challenge-2022/data?select=test_images) or use other images that are applicable 

In [14]:
#Generate files for train/validate split
import torch
import torch.utils.data as data
import os
import random
def split_data(root_data_dir, percent_used, validate_set_percent, split_config_dir, split_config_name):
    all_scenes = os.listdir(root_data_dir)
    sub_set = [scene for scene in all_scenes if random.random() <= percent_used] 
    print(sub_set)
    
    train_set_size = int(len(sub_set) * (1-validate_set_percent))
    valid_set_size = len(sub_set) - train_set_size
    
    seed = torch.Generator().manual_seed(42)
    train_set, valid_set = data.random_split(sub_set, [train_set_size, valid_set_size], generator=seed)
    print(train_set)
     
    sub_set_path = os.path.join(split_config_dir, split_config_name+'-total.txt')
    train_set_path = os.path.join(split_config_dir, split_config_name+'-train.txt')
    val_set_path = os.path.join(split_config_dir, split_config_name+'-val.txt')
    
    with open(sub_set_path, 'w') as fout:
        for scene in sub_set:
            fout.write(scene+'\n')
    with open(train_set_path, 'w') as fout:
        for scene in train_set:
            fout.write(scene+'\n')
    with open(val_set_path, 'w') as fout:
        for scene in valid_set:
            fout.write(scene+'\n')
    return train_set_path, val_set_path, sub_set_path

In [15]:
split_data('/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1', 0.025, 0.2, './assets/', 'subset-0025-02')

['0032', '0047', '0223', '0252']


('./assets/subset-0025-02-train.txt',
 './assets/subset-0025-02-val.txt',
 './assets/subset-0025-02-total.txt')

## Feature Extraction
You can run feature extraction as a separate step, prior to training. This will make training run faster, but is very expensive in terms of disk space and time, as there are not any built in options inside of the OpenGlue framework to preform feature extraction on a subset of the data. The only ways to do this are to either manually move all of the scene folders not included in the files specifying the subsets generated in the previous step to a different location, or to not run feature extraction as a separate step and instead run OpenGlue training such that feature extraction is done concurrently with training, meaning only the scene files used for training/validation will have feature extraction preformed on them.
<br />The steps to do feature extraction as a separate step are detailed in the README:<br />
run `python extract_features.py` with the following parameters (for more details, please, refer to module's documentation):
      
   * `--device` - `[cpu, cuda]`
   * `--num_workers` - number of workers for parallel processing. When cpu is chosen, assigns the exact amount of the set workers, for gpu scenario it takes into account the number of gpu units available. 
   * `--target_size` - target size of the image (WIDTH, HEIGHT).
   * `--data_path` - path to directory with scenes images
   * `--output_path` - path to directory where extracted features are stored
   * `--extractor_config_path` - 'path to the file containing config for feature extractor in .yaml format
   * `--recompute` - include this flag to recompute features if it is already present in output directory
   * `--image_format` - formats of images searched inside `data_path` to compute features for, default: [jpg, JPEG, JPG, png]
   
   Choosing local feature extractor is performed via `--extractor_config_path`. 
   We provide configs in `config/features/`, which user can edit or use unchanged:
   * SuperPoint with MagicLeap weights - [`superpoint_magicleap.yaml`](config/features/superpoint_magicleap.yaml)
   * SuperPoint with KITTI weights - [`superpoint_kitti.yaml`](config/features/superpoint_kitti.yaml)
   * SuperPoint with COCO weights - [`superpoint_coco.yaml`](config/features/superpoint_coco.yaml)
   * SIFT opencv - [`sift_opencv.yaml`](config/features/sift_opencv.yaml)
   * DoG-AffNet-HardNet - [`dog_opencv_affnet_hardnet.yaml`](config/features/dog_opencv_affnet_hardnet.yaml)
<br />I have included an example command to run this from the command line, as well as reimplemented the necessary functions such that it can be run from here, with the arguments being in a list rather than entered as command line arguments. The command line version would look like:<br /> `python extract_features.py --device 'cuda' --num_workers '1' --target_size 1280 1280 --data_path '/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1/' --output_path '/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix' --extractor_config_path 'config/features/superpoint_magicleap.yaml'`<br />
Note: if feature extraction is run as a separate step, be sure to take note of the target size you use. In this case it is 1280 by 1280, which is what I used when I ran feature extraction on it's own, but this value needs to be consistent inside the config files to be used during training. Also, this value could be made smaller, which may improve runtimes.

In [16]:
#From: extract_features.py

import argparse
import glob
import logging
import math
import pathlib
from typing import Tuple, List, Union, Optional

import cv2
import deepdish as dd
import numpy as np
import os
import torch
import yaml
from torch import multiprocessing

from models.features import get_feature_extractor

import extract_features

def parse_arguments(arguments):
    """
    Define and parse command line arguments for this module.
    Returns:
        args: Namespace object containing parsed arguments
    """
    parser = argparse.ArgumentParser('Local Features Extraction')
    parser.add_argument(
        '--device',
        help='Device type, where features are extracted',
        type=str,
        default='cpu',
        choices=['cpu', 'cuda'],
    )
    parser.add_argument(
        '--num_workers',
        help='Number of workers for parallel processing',
        type=int,
        default=1
    )
    parser.add_argument(
        '--target_size',
        help='Target size of the image (WIDTH, HEIGHT). '
             'At least one side of resulting image will correspond with `target_size` '
             'dimensions, such that the original aspect ration is preserved',
        type=int,
        default=None,
        nargs=2
    )
    parser.add_argument(
        '--data_path',
        help='Path to directory with scenes images',
        type=pathlib.Path,
        required=True
    )
    parser.add_argument(
        '--output_path',
        help='Path to directory where extracted features are stored',
        type=pathlib.Path,
        required=True
    )
    parser.add_argument(
        '--extractor_config_path',
        help='Path to the file containing config for feature extractor in .yaml format',
        type=pathlib.Path,
        default='config/features/sift_opencv.yaml'
    )
    parser.add_argument(
        '--recompute',
        help='Flag indicating whether to recompute features if it is already present in output directory',
        action='store_true'
    )
    parser.add_argument(
        '--image_format',
        help='Formats of images searched inside `data_path` to compute features for',
        type=str,
        default=['jpg', 'JPEG', 'JPG', 'png'],
        nargs='+'
    )
    parser.add_argument(
        '--scene_subset_path',
        help='Path to the file containing the list of scenes to preform feature extraction on within the root directory. All scenes proccessed if unset.',
        type=pathlib.Path,
        default='./None'
    )
    return parser.parse_args(arguments)

def get_images_list(input_data_path: pathlib.Path, image_formats: List[str], scene_subset_path: pathlib.Path) -> List[Tuple[str, Optional[str]]]:
    """
    Get list of images that wil be processed.
    Args:
        input_data_path: input path to the location with images
        image_formats: file formats of images to look for
        scence_subset_path: input path to file specifying which scenes to use. All scenes used if set to ./None
    Returns:
        images_path_list: list where each image is represented as tuple with path to image
        and scene name (or None if no scenes are available)
    """
    
    scene_set = []
    if scene_subset_path != "./None":
        with open(scene_subset_path, 'r') as fin:
            scene_set = fin.read().split('\n')
    print(scene_set)
    # process each scene
    images_path_list = []
    
    #Added to take subset prior to extraction
    scenes = os.listdir(input_data_path)
    for scene in scenes:
        if scene not in scene_set:
            continue
            
            
        images_path = input_data_path / scene / 'dense0' / 'imgs'
        scene_list = []
        for image_format in image_formats:
            scene_list.extend(glob.glob(str(images_path / f'*.{image_format}')))
        images_path_list.extend(((path, scene) for path in scene_list))
    return images_path_list

def extract(arguments):
    logger = logging.getLogger(__name__)
    args = parse_arguments(arguments)
    logger.info(args)

    num_workers = args.num_workers
    if args.device == 'cuda' and torch.cuda.device_count() < num_workers:
        logger.warning(f'Number of workers selected is bigger than number of available cuda devices. '
                       f'Setting num_workers to {torch.cuda.device_count()}.')
        num_workers = torch.cuda.device_count()

    # read feature extractor config
    with open(args.extractor_config_path) as f:
        feature_extractor_config = yaml.full_load(f)

    # make output directory
    output_path = args.output_path / extract_features.get_output_directory_name(feature_extractor_config, args)
    logger.info(f'Creating output directory {output_path} (if not exists).')
    os.makedirs(output_path, exist_ok=True)
    with open(os.path.join(output_path, 'config.yaml'), 'w') as f:
        yaml.dump(feature_extractor_config, f)


    
    images_list = get_images_list(args.data_path, args.image_format, args.scene_subset_path)
    logger.info(f'Total number of images found to process: {len(images_list)}')
    # split into chunks of (almost) equal size
    chunk_size = math.ceil(len(images_list) / num_workers)
    images_list = [images_list[i * chunk_size:(i + 1) * chunk_size] for i in range(num_workers)]

    logger.info(f'Starting {num_workers} processes for features extraction.')
    multiprocessing.start_processes(
        extract_features.process_chunk,
        args=(images_list, feature_extractor_config, output_path, args),
        nprocs=num_workers,
        join=True
    )

In [ ]:
args = ['--device', 'cuda', '--num_workers', '1', '--target_size', '960', '720', '--data_path', '/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1/', '--output_path', '/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix3', '--extractor_config_path', './config/features/superpoint_magicleap_1024.yaml', '--scene_subset_path', './assets/subset-20-20-total.txt']
extract(args)


[2022/06/13 11:14:54] __main__ | INFO: Namespace(data_path=PosixPath('/host_Data/Data/MegaDepth/MegaDepth/phoenix/S6/zl548/MegaDepth_v1'), device='cuda', extractor_config_path=PosixPath('config/features/superpoint_magicleap.yaml'), image_format=['jpg', 'JPEG', 'JPG', 'png'], num_workers=1, output_path=PosixPath('/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix2'), recompute=False, scene_subset_path=PosixPath('assets/subset-20-20-total.txt'), target_size=[960, 720])
[2022/06/13 11:14:54] __main__ | INFO: Creating output directory /host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix2/SuperPointNet_960_720 (if not exists).


['0024', '0048', '0102', '0148', '0181', '0307', '']
['0000', '0001', '0002', '0003', '0004', '0005', '0007', '0008', '0011', '0012', '0013', '0015', '0016', '0017', '0019', '0020', '0021', '0022', '0023', '0024', '0025', '0026', '0027', '0032', '0033', '0034', '0035', '0036', '0037', '0039', '0041', '0042', '0043', '0044', '0046', '0047', '0048', '0049', '0050', '0056', '0057', '0058', '0060', '0061', '0062', '0063', '0064', '0065', '0067', '0070', '0071', '0076', '0078', '0080', '0083', '0086', '0087', '0090', '0092', '0094', '0095', '0098', '0099', '0100', '0101', '0102', '0103', '0104', '0105', '0107', '0115', '0117', '0121', '0122', '0129', '0130', '0133', '0137', '0141', '0143', '0147', '0148', '0149', '0150', '0151', '0156', '0160', '0162', '0168', '0175', '0176', '0177', '0178', '0181', '0183', '0185', '0186', '0189', '0190', '0197', '0200', '0204', '0205', '0209', '0212', '0214', '0217', '0223', '0224', '0229', '0231', '0235', '0237', '0238', '0240', '0243', '0252', '0257', '0

[2022/06/13 11:14:55] __main__ | INFO: Total number of images found to process: 2147
[2022/06/13 11:14:55] __main__ | INFO: Starting 1 processes for features extraction.


<All keys matched successfully>


[2022/06/13 11:15:34] extract_features | INFO: PID #0: Processed 100/2147 images.


## Training
Before training, set all necessary hyperparameters configurations in your config file.
If you selected to extract features during training, then edit [`config/config.yaml`](config/config.yaml). For pre-extracted features `config/config_cached.yaml` will be used.


The description of possible configurations for training is provided in [`CONFIGURATIONS.md`](CONFIGURATIONS.md)


Be aware, that default configs for feature extraction set a maximum of 1024 keypoints per image. For second option - pre-extraction, more keypoints can be detected and saved.
When calculating features during training, one of the corresponding configs from [`config/features_online`](config/features_online) should be selected and forwarded as a parameter, see example below. So, there is a great difference between launching with config.yaml or config_cached.yaml .


<b>To launch train with local feature extraction throughout training, run: </b>  
```
python train.py --config='config/config.yaml' --features_config='config/features_online/sift.yaml'
```
sift.yaml is an example, can be any config file from `config/features_online`. <i>Important Note: </i> DoG-AffNet-HardNet is only available for cached features run. We do not recommend extracting features with this model during training.


<b>To launch train with cached local features, run: </b>   
```
python train_cached.py --config='config/config_cached.yaml'
```
The parameter responsible for configuring cached features location is `features_dir`.


The logging results will be visible inside a log folder + experiment name, specified in `config.yaml`. <br />

Given the plugins and training settings that OpenGlue uses, specifically the accelerator and strategy settings, training cannot be run inside of a jupyter notebook. Run the given command, depending on whether you are using pre-extracted features or not, from the command line.

## Training with pre-extraction (cached)

In [1]:
#

import torch
import shutup

shutup.please()
import os
import argparse
from datetime import datetime
from omegaconf import OmegaConf
import pytorch_lightning as pl
from pytorch_lightning.strategies import DataParallelStrategy

from data.megadepth_datamodule import MegaDepthPairsDataModuleFeatures
from models.matching_module import MatchingTrainingModule
from utils.train_utils import get_training_loggers, get_training_callbacks, prepare_logging_directory


def train_cached(config_path='config/config_cached.yaml'):

    # Load config
    config = OmegaConf.load('config/config_cached.yaml')  # base config
    if config_path != 'config/config_cached.yaml':
        add_conf = OmegaConf.load(config_path)
        config = OmegaConf.merge(config, add_conf)

    pl.seed_everything(int(os.environ.get('LOCAL_RANK', 0)))

    # Prepare directory for logs and checkpoints
    if os.environ.get('LOCAL_RANK', 0) == 0:
        experiment_name = '{}_cache__attn_{}__laf_{}__{}'.format(
            config['data']['features_dir'],
            config['superglue']['attention_gnn']['attention'],
            config['superglue']['laf_to_sideinfo_method'],
            str(datetime.now().strftime("%Y-%m-%d-%H-%M-%S"))
        )
        log_path = prepare_logging_directory(config, experiment_name)
    else:
        experiment_name, log_path = '', ''

    # Init Lightning Data Module
    data_config = config['data']
    dm = MegaDepthPairsDataModuleFeatures(
        root_path=data_config['root_path'],
        train_list_path=data_config['train_list_path'],
        val_list_path=data_config['val_list_path'],
        test_list_path=data_config['test_list_path'],
        batch_size=data_config['batch_size_per_gpu'],
        num_workers=data_config['dataloader_workers_per_gpu'],
        target_size=data_config['target_size'],
        features_dir=data_config['features_dir'],
        num_keypoints=data_config['max_keypoints'],
        val_max_pairs_per_scene=data_config['val_max_pairs_per_scene'],
        balanced_train=data_config.get('balanced_train', False),
        train_pairs_overlap=data_config.get('train_pairs_overlap')
    )

    features_config = OmegaConf.load(os.path.join(config['data']['root_path'],
                                                  config['data']['features_dir'], 'config.yaml'))

    # Init model
    model = MatchingTrainingModule(
        train_config={**config['train'], **config['inference'], **config['evaluation']},
        features_config=features_config,
        superglue_config=config['superglue'],
    )

    # Set callbacks and loggers
    callbacks = get_training_callbacks(config, log_path, experiment_name)
    loggers = get_training_loggers(config, log_path, experiment_name)

    # Init trainer
    trainer = pl.Trainer(
        gpus=config['gpus'],
        max_epochs=config['train']['epochs'],
        accelerator="gpu",
        gradient_clip_val=config['train']['grad_clip'],
        log_every_n_steps=config['logging']['train_logs_steps'],
        limit_train_batches=config['train']['steps_per_epoch'],
        num_sanity_val_steps=5,
        callbacks=callbacks,
        logger=loggers,
        strategy=DataParallelStrategy(),
        #plugins=DDPPlugin(find_unused_parameters=False),
        precision=config['train'].get('precision', 32),
    )
    # If loaded from checkpoint - validate
    if config.get('checkpoint') is not None:
        trainer.validate(model, datamodule=dm, ckpt_path=config.get('checkpoint'))
    trainer.fit(model, datamodule=dm, ckpt_path=config.get('checkpoint'))


/root/code/OpenGlue/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.cuda.device_count())
!nvcc --version
!python --version

print(torch._C._cuda_getCompiledVersion())
print(torch._C._cuda_getDevice())
print(torch._C._cuda_getDeviceCount())

1
nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2021 NVIDIA Corporation
Built on Sun_Feb_14_21:12:58_PST_2021
Cuda compilation tools, release 11.2, V11.2.152
Build cuda_11.2.r11.2/compiler.29618528_0
Python 3.8.10
11030
0
1


In [ ]:
train_cached()

Global seed set to 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jlunder. Use `wandb login --relogin` to force relogin


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name                   | Type                      | Params
---------------------------------------------------------------------
0 | superglue              | SuperGlue                 | 12.0 M
1 | augmentations          | AugmentationSequential    | 0     
2 | epipolar_dist_metric   | AccuracyUsingEpipolarDist | 0     
3 | camera_pose_auc_metric | CameraPoseAUC             | 0     
---------------------------------------------------------------------
12.0 M    Trainable params
0         Non-trainable params
12.0 M    Total params
47.829    Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/5 [00:00<?, ?it/s]correction?
correction?
correction?
Sanity Checking DataLoader 0:  20%|██        | 1/5 [00:00<00:00, 11.81it/s]correction?
correction?
correction?
Sanity Checking DataLoader 0:  40%|████      | 2/5 [00:00<00:01,  2.20it/s]correction?
correction?
correction?
Sanity Checking DataLoader 0:  60%|██████    | 3/5 [00:01<00:01,  1.53it/s]correction?
correction?
correction?
Sanity Checking DataLoader 0:  80%|████████  | 4/5 [00:02<00:00,  1.29it/s]correction?
correction?
correction?
Sanity Checking DataLoader 0: 100%|██████████| 5/5 [00:03<00:00,  1.27it/s]correction?
correction?
correction?
Epoch 0:  91%|█████████ | 500/550 [03:34<00:21,  2.34it/s, loss=3.44, v_num=o6qt, Train NLL loss=2.670, Train Metric loss=0.000]
Validation: 0it [00:00, ?it/s]
Validation DataLoader 0:   0%|          | 0/50 [00:00<?, ?it/s]correction?
correction?
correction?

Epoch 0:  91%|█████████ | 501/550 [03:34<00:21,  2.33it/s, loss=3.44, v_num=o6qt, 

## Inference

In [3]:

import inference
import matplotlib.pyplot as plt

import kornia as K
import kornia.feature as KF
from kornia_moons.feature import *

import torch
import torch.nn as nn

#Where the images to run inference on are
img0_path = '/host_Data/Data/MegaDepth/MegaDepth/test_images/img2.jpg'
img1_path = '/host_Data/Data/MegaDepth/MegaDepth/test_images/img3.jpg'

#Where the configurations and checkpoints saved during training are
experiment_path = '/host_Data/Data/MegaDepth/MegaDepth/extracted_features/phoenix3/SuperPointNet_960_720_cache__attn_softmax__laf_none__2022-06-14-21-21-28'
checkpoint_name = 'superglue-step=1000.ckpt'

#which device to use to run inference (gpu or cpu)
device = 'cuda:0'

#output location
output_dir = 'results.png'

- Config.yaml in 'experiment_path' directory
    - Under superglue:
        - Add 'descriptor_dim: <dim>' where <dim> is the same as the corresponding field in features_config.yaml in the same folder (usually 256)
        - Under positional_encoding:
            - Add and set 'output_size: ' to the same value
        - Under attention_gnn:
            - Add and set 'embed_dim: ' to the same value
   
   

In [4]:
img0, img1, lafs0, lafs1, inliers = inference.run_inference(img0_path, img1_path, experiment_path, checkpoint_name, device)
draw_LAF_matches(
    lafs0,
    lafs1,
    torch.arange(len(inliers)).view(-1, 1).repeat(1, 2),
    K.tensor_to_image(img0),
    K.tensor_to_image(img1),
    inliers,
    draw_dict={'inlier_color': (0.2, 1, 0.2),
               'tentative_color': None,
               'feature_color': (0.2, 0.5, 1), 'vertical': True})
plt.savefig(output_dir)

{'descriptor_dim': 256, 'name': 'SuperPointNet', 'parameters': {'descriptor_dim': 256, 'keypoint_threshold': 0.005, 'max_keypoints': 1024, 'nms_kernel': 9, 'remove_borders_size': 4, 'weights': './models/superglue/weights/superpoint_magicleap_v1.pth'}, 'max_keypoints': 1024}
<All keys matched successfully>
superglue.mix_coefs
superglue.dustbin_score
superglue.positional_encoding.encoder.0.weight
superglue.positional_encoding.encoder.0.bias
superglue.positional_encoding.encoder.2.weight
superglue.positional_encoding.encoder.2.bias
superglue.positional_encoding.encoder.2.running_mean
superglue.positional_encoding.encoder.2.running_var
superglue.positional_encoding.encoder.2.num_batches_tracked
superglue.positional_encoding.encoder.3.weight
superglue.positional_encoding.encoder.3.bias
superglue.positional_encoding.encoder.5.weight
superglue.positional_encoding.encoder.5.bias
superglue.positional_encoding.encoder.5.running_mean
superglue.positional_encoding.encoder.5.running_var
superglue.p

RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor

In [7]:
%load_ext autoreload

In [12]:
%autoreload